In [ ]:
# Final Project for GEOG 650 at Libetry University
# Written by David J. Anderson
# 6 March 2025

# Import Libraries and generate user output when each import is complete
import arcpy
from arcpy.sa import *
print("arcpy has loaded properly.")

# Set up geoprocessing environment
arcpy.env.workspace = r"C:\Lessons\FinalProject"
arcpy.env.overwriteOutput = True

# Build a list of rasters in the Final Projet directory to merge into the new raster file
Rasterlist = arcpy.ListRasters()

# Perform Mosaic to New Raster to combine multiple mosaic DEM files into a single raster
# Using the Rasterlist as an input, run the .arcpy.management.MosaicToNewRaster 
arcpy.management.MosaicToNewRaster(
    input_rasters=Rasterlist,
    output_location=r"C:\Lessons\FinalProject",
    raster_dataset_name_with_extension="SanJuanIslandsWGS84.tif",
    coordinate_system_for_the_raster='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
    pixel_type="8_BIT_UNSIGNED",
    cellsize=None,
    number_of_bands=1,
    mosaic_method="LAST",
    mosaic_colormap_mode="FIRST"
)

# Change the raster projection to match the polygon features used later for the mask
arcpy.management.ProjectRaster(
    in_raster="SanJuanIslandsWGS84.tif",
    out_raster=r"C:\Lessons\FinalProject\GIS_650_Final_Project\GIS_650_Final_Project.gdb\SanJuanIslands_FinalRaster",
    out_coor_system='PROJCS["NAD_1983_StatePlane_Washington_North_FIPS_4601_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",1640416.666666667],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-120.8333333333333],PARAMETER["Standard_Parallel_1",47.5],PARAMETER["Standard_Parallel_2",48.73333333333333],PARAMETER["Latitude_Of_Origin",47.0],UNIT["Foot_US",0.3048006096012192]]',
    resampling_type="NEAREST",
    cell_size="124.632298127012 124.632298127012",
    geographic_transform="WGS_1984_(ITRF00)_To_NAD_1983",
    Registration_Point=None,
    in_coor_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
    vertical="NO_VERTICAL"
)

# Run a Raster Calculator function to only select areas higher in elevation than sea level
input_raster = arcpy.Raster(r"C:\Lessons\FinalProject\GIS_650_Final_Project\GIS_650_Final_Project.gdb\SanJuanIslands_FinalRaster")
output_raster = input_raster > 0
output_raster.save(r"C:\Lessons\FinalProject\GIS_650_Final_Project\GIS_650_Final_Project.gdb\Land_Area")

# Clip the raster to the extent of the Orcas Islands
arcpy.management.Clip(
    in_raster="output_raster",
    rectangle="1103201.95624238 585371.718816146 1179200.56638399 659907.925386474",
    out_raster=r"C:\Lessons\FinalProject\GIS_650_Final_Project\GIS_650_Final_Project.gdb\output_raster_Clip",
    in_template_dataset="Orcas_Islands",
    nodata_value="255",
    clipping_geometry="ClippingGeometry",
    maintain_clipping_extent="MAINTAIN_EXTENT"
)